In [4]:
# ex06_mlp_

%cd 'drive/MyDrive/Colab Notebooks/23.12.18딥러닝'

/content/drive/MyDrive/Colab Notebooks/23.12.18딥러닝


### 목표
- 이미지 형태의 데이터를 이해해보자
- 활성화 함수, 최적화 함수 비교해보자
- 딥러닝 다중분류 모델을 학습시켜보자
- 최적의 모델을 저장, 조기학습 중단까지 연결해보자
- 저장된 모델을 불러와서 사용하자

In [5]:
# 환경셋팅
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.datasets import fashion_mnist


In [6]:
# 데이터 담기
data=fashion_mnist.load_data()

In [ ]:
data # 3차원 data : ((X_train,y_train),(X_test,y_test))
X_train=data[0][0]
y_train=data[0][1]
X_test=data[1][0]
y_test=data[1][1]
# (X_train,y_train),(X_test,y_test)=fashion_mnist.load_data()
print(X_train,y_train)
print(X_test,y_test)
# (이미지 개수, 행, 열),(답)

In [18]:
data[0][0]
data[0][1]
data[1][0]
data[1][1]


array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [ ]:
# 이미지 한장 출력해보기
# X_train 100번째 인덱스에 해당하는 이미지 접근하기
for i in range(1,31):

  img100=X_train[i]
  plt.imshow(img100, cmap='gray')
  plt.show()

In [ ]:
random_list=np.random.randint(50000,size=30)
random_list

for idx in random_list :
  plt.imshow(X_train[idx],cmap='gray')
  plt.show()

In [21]:
# y_train
# 답데이터의 카테고리 개수

# 마지막 출력층의 유닉을 결정

# 각각 카테고리는 몇개씩 들어있는지
np.unique(y_train)
np.unique(y_train, return_counts=True)


(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000]))

In [23]:
# 다중분류 처리 위해 답 데이터 원핫 인코딩
# pd-> pd.get_dummies(y_train)>> 판다스 df
# keras.untils-> to_categorical(y_train)>> 넘파이 arr
y_train_oh=pd.get_dummies(y_train)
y_test_oh=pd.get_dummies(y_test)

y_train_oh.shape,y_test_oh.shape

((60000, 10), (10000, 10))

In [25]:
# Flatten 가정
# 문제데이터 전처리
# mlp 1차원만 인식 가능
# 이미지 1장 3차원 > 1차원 펴줌
X_train2=X_train.reshape(60000,28*28)
X_test2=X_test.reshape(10000,28*28) # X_test.reshape(10000,-1) 데이터의 크기에 맞춰서 알아서 펴줌
print('변형된 훈련문제 : ',X_train2.shape)
print('변형된 테스트문제 : ',X_test2.shape)
# 이미지 한장 1차원, 784개의 특성을 가지고 있음

변형된 훈련문제 :  (60000, 784)
변형된 테스트문제 :  (10000, 784)


### 활성화 함수, 최적화 함수별 비교
- 1. 중간층의 활성화 : sigmoid, 최적화 함수:sgd
- 2. 중간층 활성화 : relu,최적화 함수:sgd
- 3. 중간층 활성화 : relu,최적화 함수:adam
  - 각 신경망을 설계하고 결과 비교

In [28]:
from tensorflow.keras.optimizers import SGD,Adam

In [37]:
# 모델생성함수
# 활성화 함수, 최적화 함수 매개변수로 연결
def create_model(acti,opti):
  # 모델 생성
  model=Sequential()
  # 중간층 2개(100,50)
  model.add(Dense(units=200,input_dim=784,activation=acti))
  model.add(Dense(100,activation=acti))
  model.add(Dense(50,activation=acti))
  model.add(Dense(10,activation="softmax"))

  # 출력층 (클래스 개수만큼, 활성화='softmax')

  # 학습/평가 방법 설정(compile)
  # loss,optimizer,metrics=['accuracy']
  model.compile(loss="categorical_crossentropy",
                optimizer=opti,
                metrics=['accuracy'])
  return model

In [40]:
# fit 함수 정의
# 예측기 estimator
# 모델명만 넣어주면 fit하는 함수 생성
def fit_model(estimator):
  # 학습 코드 : 30번 반복, 문제, 답, 검증 데이터(train 데이터 분리)
  h = estimator.fit(X_train2,y_train_oh,epochs=30,validation_split=0.3)
  #return 학습현황
  return h

In [ ]:
# 학습을 조정하고 싶다면 SGD불러서 변경 가능
# 'sgd',SGD (leanrning_rate=)
model1=create_model(acti='sigmoid',opti=SGD(learning_rate=0.01))
h1=fit_model(model1)

In [ ]:
# 2. relu+sgd(0.01)
model2=create_model(acti='relu',opti=SGD(learning_rate=0.01))
h2=fit_model(model2)

In [ ]:
# 3.model3 : 'relu'+Adam(0.001)
model3=create_model(acti='relu',opti=Adam(learning_rate=0.01))
h3=fit_model(model3)

In [ ]:
# h1,h2,h3학습현황 활용해서 선 그래프 시각화
# accuracy,val_accuracy -6개 선 그래프 그릴것
# 범례 출력
# 시각화를 통해서 어떤 활성화 , 최적화 조합이 좋았는지 비교해보기
plt.figure(figsize=(15,4))
plt.plot(h1.history['accuracy'],label='h1 sidsgd')
plt.plot(h1.history['val_accuracy'],label='h1 val_sidsgd')


plt.plot(h2.history['accuracy'],label='h2 sidsgd')
plt.plot(h2.history['val_accuracy'],label='h2 val_resgd')

plt.plot(h3.history['accuracy'],label='h3 sidsgd')
plt.plot(h3.history['val_accuracy'],label='h3 val_reAd')

#### 최적화 모델 찾기
- 모델 체크 포인트: 최적의 성능일때의 모델저장
- 얼리스탑핑 : 조기 학습중단

In [48]:
# os 라이브러리 임폴트 - 운영체제
# 폴더 생성하는 기능 사용
import os
# mckp, es
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [60]:
# 모델 저장 관련 설정
# 1. 저장할 폴더 생성
model_dir='./model'
# 없으면 생성
if not os.path.exists(model_dir):
  os.mkdir(model_dir)

# 2. 어떤 상태의 모델을 저장할건지, 어떤 경로에 저장할 건지 정의
# fashion_model_01_0.264-8.hdf5
# {매개변수:저장형식}:정규표현법(식)
file_path = model_dir + 'fashion_model_{epoch:02d}_{val_accuracy:.3f}.hdf5'
f_mckp=ModelCheckpoint(filepath=file_path, # 모델 저장 경로
                       monitor='val_accuracy', # 모니터 할 값
                       save_best_only=True, # 저장 시점 - 성능이 개선 됐응ㄹ때만
                       verbose=1) # 저장되는 현황 문자열 출력


In [61]:
# 조기학습 중단 정의
# 어떤 상태일때 학습을 중단할 건지 설정
f_early=EarlyStopping(monitor='val_accuracy',# 모니터 할 값
                      patience=5, # 몇번 성능개선을 기다릴지 정하는 횟수
                      verbose=1) # 조기 학습중단 현황 문자열 출력

In [ ]:
# 모델 학습
# 저장, 멈추는 기능 연결
model4=create_model('relu','adam')
h4=model4.fit(X_train2,y_train_oh,
              validation_split=0.3,
              epochs=30,
              callbacks=[f_mckp,f_early])

In [65]:
#modelfashion_model_11_0.877.hdf5
# tf 모델 불러오기
from tensorflow.keras.models import load_model

In [68]:
best_model = load_model('./modelfashion_model_11_0.877.hdf5')
best_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 200)               157000    
                                                                 
 dense_45 (Dense)            (None, 100)               20100     
                                                                 
 dense_46 (Dense)            (None, 50)                5050      
                                                                 
 dense_47 (Dense)            (None, 10)                510       
                                                                 
Total params: 182660 (713.52 KB)
Trainable params: 182660 (713.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [69]:
# test2 데이터 활용하기
# 예측 predict
# 평가 evalue
pred=best_model.predict(X_test2)
pred.shape

313/313 [==============================] - 1s 2ms/step


(10000, 10)

In [73]:
pd.Series(pred[0]).sort_values(ascending=False)
# 9라는 클래스로 예측함

9    9.645669e-01
7    2.854812e-02
5    6.884967e-03
8    2.116787e-09
1    5.096487e-14
3    2.623753e-17
6    2.563421e-19
4    2.004160e-21
2    1.987000e-23
0    4.984839e-26
dtype: float32

In [74]:
y_test[0]

9

In [75]:
best_model.evaluate(X_test2,y_test_oh)

313/313 [==============================] - 2s 5ms/step - loss: 0.3891 - accuracy: 0.8663


[0.38911867141723633, 0.8662999868392944]